# Tratamento dos dados
O objetivo desse notebook é fazer um pré-processamento dos dados que permita sua utilização na modelagem.

As etapas passam desde formatação de campos, remoção colunas com muitos valores faltantes, conversão de categorias em features  e alguns outros tratamentos.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_selection import VarianceThreshold
from sklearn.utils import shuffle
from collections import Counter

## Listings
Como este arquivo possui muitas colunas, foi feito um primeiro filtro manual para remover as colunas que não seriam exploradas nesta análise.

Portanto, criei um arquivo chamado listing_columns.csv que contém apenas as colunas que valem a pena serem estudadas e em seguida substitui o arquivo original com as colunas filtradas.

Assim, é mais fácil de lidar com o arquivo, além de permitir que eu faça o push do repositório no git.

Removi também outras variáveis relacionadas à preço:
- weekly_price
- monthly_price
- cleaning_fee
- security_deposit
Isso foi feito pois estou assumindo que o modelo desenvolvido não terá acesso à essas informações a priori, o que poderia enviesar minha análise

In [2]:
listings_columns_file = './../dados_processados/listing_columns.csv'
listings_file = './../dados_brutos/listings_detailed.csv'

In [3]:
listings_columns = pd.read_csv(listings_columns_file,
                               sep=';'
                              ).columns

In [4]:
listings = pd.read_csv(listings_file,
                       usecols=listings_columns,
                       low_memory=False)

In [5]:
listings.head()

,id,experiences_offered,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,17878,none,2010-01-08,within an hour,100%,NaN,t,Copacabana,2.0,2.0,...,t,f,strict_14_with_grace_period,f,f,1,1,0,0,2.12
1,21280,none,2010-02-14,within an hour,100%,NaN,f,Ipanema,0.0,0.0,...,f,f,strict_14_with_grace_period,f,f,1,1,0,0,1.23
2,25026,none,2010-04-03,within a day,86%,NaN,f,Copacabana,3.0,3.0,...,f,f,strict_14_with_grace_period,t,t,3,3,0,0,2.02
3,31560,none,2010-05-31,within an hour,100%,NaN,t,Ipanema,1.0,1.0,...,t,f,strict_14_with_grace_period,f,f,1,1,0,0,2.39
4,35636,none,2010-06-27,within an hour,100%,NaN,f,Ipanema,1.0,1.0,...,f,f,strict_14_with_grace_period,t,t,1,1,0,0,2.29


## Arruma formato da nossa variável dependente Preço

In [6]:
listings['price'] = listings['price'].str.replace('$','').str.replace(',','').astype(float)

## Filtra apenas Preços maiores que zero

In [7]:
listings = listings.loc[listings['price']>0]

## Remove colunas com muitos valores faltantes
Regra adotada: Campos que possuem mais de 30% de missing values não valem a pena serem utilizadas na modelagem

In [8]:
length = listings.shape[0]
valid_cols = (listings.isnull().sum() / length < 0.30)
print('Numero de colunas validas: {}'.format(valid_cols.sum()))

Numero de colunas validas: 45


In [9]:
listings = listings.loc[:,valid_cols]

## Tratamento de variáveis categóricas

### Remove colunas sem informação

In [10]:
print('Colunas sem informação:')
for c in listings.columns:
    if listings.loc[:,c].nunique() == 1:
        print(c)
        listings.drop(c,axis=1,inplace=True)

Colunas sem informação:
experiences_offered
has_availability
requires_license
is_business_travel_ready


In [11]:
listings.loc[:,listings.dtypes == 'object'].head()

,host_since,host_is_superhost,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,is_location_exact,property_type,room_type,bed_type,amenities,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
0,2010-01-08,t,t,t,Copacabana,t,Condominium,Entire home/apt,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",t,strict_14_with_grace_period,f,f
1,2010-02-14,f,t,t,Ipanema,t,Apartment,Entire home/apt,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",f,strict_14_with_grace_period,f,f
2,2010-04-03,f,t,t,Copacabana,t,Apartment,Entire home/apt,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",f,strict_14_with_grace_period,t,t
3,2010-05-31,t,t,t,Ipanema,t,Apartment,Entire home/apt,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",t,strict_14_with_grace_period,f,f
4,2010-06-27,f,t,t,Ipanema,t,Apartment,Entire home/apt,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",f,strict_14_with_grace_period,t,t


### Variável host_since
Vou transformar a variável host_since de forma que ela seja utilizável.

A ideia é verificar a hipótese de que o período desde que o usúario é um anfitrião possa ter relação com o preço 

In [12]:
listings['host_since'] = listings['host_since'].str[0:4].astype(float)

### Variáveis binárias (True / False)
Foi feito um mapeamento de quais variáveis são binárias e estão no formato texto.

In [13]:
binary_cols = ['host_is_superhost','host_has_profile_pic','host_identity_verified',
               'is_location_exact','instant_bookable','require_guest_profile_picture',
               'require_guest_phone_verification'
              ]
for c in listings[binary_cols]:
    listings[c] = listings[c].apply(lambda x: 1 if x=='t' else 0)

### Converte a coluna amenities em multiplas colunas
Seria interessante analisar se as facilidades descritas pelo anunciante podem impactar no preço.

Portanto, foi feito um tratamento para separar essa coluna "amenities" em multiplas colunas

In [14]:
def string_dict_to_dict(x):
    """
    Transforma o dicionario em string para um dicionario real
    """
    y = list(eval(x.replace('"','').replace('{','{"').replace('}','"}').replace(',','","')))
    
    return y

In [15]:
# Será feito algo similar à um bag of words, porém listando todas as amenities possiveis. 
list_of_amenities = []
for idx,row in listings.iterrows():
    y = string_dict_to_dict(row['amenities'])
    list_of_amenities.append(y)
amenities = [item for sublist in list_of_amenities for item in sublist]

# Considera apenas as top 20 facilidades
amenities = list(dict(Counter(amenities).most_common(20)).keys())
for c in amenities:
    listings[c] = listings['amenities'].apply(lambda x: c in x).astype(int)
listings.drop('amenities',axis=1,inplace=True)

### One Hot Encoding das variáveis categóricas restantes

In [16]:
categorical_cols = ['property_type', 'room_type', 'bed_type', 'cancellation_policy']
dummies = pd.get_dummies(listings.set_index('id')[categorical_cols],drop_first=True)
listings = listings.drop(categorical_cols,axis=1)
listings = listings.merge(dummies,how='left',on='id')

## Separar conjunto de treino e de teste
Será utilizado 70% para treino/validação e 30% para teste

In [17]:
train_set = listings.sample(frac=0.7)
test_set = listings.drop(train_set.index)

In [18]:
train_set.to_csv('./../dados_processados/train_sample.csv',index=False)
test_set.to_csv('./../dados_processados/test_sample.csv',index=False)

## Com base na conjunto de treinamento, vamos extrair algumas informações dos bairros
Utilizo apenas o conjunto de treinamento para não enviesar os resultados obtidos quando validar meu modelo no conjunto de teste.

A utilização dessas features parte da premissa de que essas informações extraidas das amostras são estatisticamente significantes para explicar caracteristicas do bairro e, portanto, podem ser usadas na modelagem.

A partir do número de anúncios em cada bairro (neighbourhood_clean), calculamos:
- Distribuição dos anuncios no Rio de Janeiro
- Preço médio em cada bairro

In [19]:
densidade_bairro =  train_set.groupby(['neighbourhood_cleansed']
                                     )[['id']].count() / train_set.shape[0]

preco_medio_bairro = train_set.groupby(['neighbourhood_cleansed']
                                      )[['price']].mean()

info_bairro = densidade_bairro.merge(preco_medio_bairro,
                                     on='neighbourhood_cleansed'
                                    ).reset_index()

info_bairro.rename(
    columns={'id':'neighbourhood_density',
             'price':'neighbourhood_mean_price'},
    inplace=True
)

### Salva arquivo para ser utilizado futuramente

In [20]:
info_bairro.to_csv('./../dados_processados/neighbourhood_info.csv',index=False)